In [13]:
from riotwatcher import ValWatcher, ApiError, RiotWatcher, LolWatcher
import pandas as pd
# global variables
api_key = 'RGAPI-37049464-7e67-4cb6-99cc-3fd9aae9564e' # API Key Accessed from Riot Dev Portal 
ValorantWatcher = ValWatcher(api_key)
RiotWatcher = RiotWatcher(api_key)
LeagueWatcher = LolWatcher(api_key)

In [9]:
#Function gets player's Player Universally Unique Identifier by using their Summoner/Game Name and Tagline.
def get_puuid(Region,GameName,TagLine):

    PUUID = RiotWatcher.account.by_riot_id(Region,GameName,TagLine)['puuid']
    
    return PUUID

In [10]:
get_puuid('AMERICAS','Valantine','Love')

'skDPBeoACRs-Y0VMzH3xsfMb7NuBWouyXmAE2HIPYda54S_YSoD51uYtWTEyXFhXEW3UVQypBe97ZQ'

In [37]:
def Get_Summoner_Info(Region,Summoner_Name):
    Summoner = LeagueWatcher.summoner.by_name(Region,Summoner_Name)
    
    return Summoner

In [80]:
Get_Summoner_Info('NA1','Doublelift')

{'id': '3TGCaO02K-kW7KSkna-MPOte7TDkoS7N8wEEYNpedfn8cyM',
 'accountId': 'Y9kprw7gtkpnnxpOLCHRkXQFPAa0KwjcU4qFColKz8crjw',
 'puuid': 'BhEiz8YXoxZj4KedTLr-puND9PmFQvqUO7P8dkpeIJ-YAOyIrTTn9LfIRP_Ota2Pp9nULawNxuYlMQ',
 'name': 'Doublelift',
 'profileIconId': 2076,
 'revisionDate': 1680815322000,
 'summonerLevel': 386}

In [217]:
def Get_Summoner_id(Region,Summoner_Name):
    Summoner = LeagueWatcher.summoner.by_name(Region,Summoner_Name)['id']
    
    return Summoner

In [295]:
Get_Summoner_id('NA1','Doublelift')

'3TGCaO02K-kW7KSkna-MPOte7TDkoS7N8wEEYNpedfn8cyM'

In [316]:
def Get_Ranked_Stats(Region,Summoner):
    Ranked_Stats = LeagueWatcher.league.by_summoner(Region, Summoner['id'])
    if(Ranked_Stats==[]):
        Temp = None
    else:
        Ranked_Stats.sort()
        Temp = Ranked_Stats[0]
    return Temp

In [317]:
Get_Ranked_Stats('NA1',Get_Summoner_Info('NA1','Doublelift'))

{'leagueId': '1c4447d2-69df-4d95-929b-6f8a3a4a2877',
 'queueType': 'RANKED_SOLO_5x5',
 'tier': 'DIAMOND',
 'rank': 'I',
 'summonerId': '3TGCaO02K-kW7KSkna-MPOte7TDkoS7N8wEEYNpedfn8cyM',
 'summonerName': 'Doublelift',
 'leaguePoints': 75,
 'wins': 92,
 'losses': 96,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': True}

In [312]:
def Get_Rank(Region,SummonerID):
    Ranked_Stats = LeagueWatcher.league.by_summoner(Region, SummonerID)
    if(Ranked_Stats==[]):
        Temp = None
    else:
        Ranked_Stats.sort()
        Temp = Ranked_Stats[0]
    if (Temp== None):
        Rank = 'Unranked'
    else:
        Rank = Temp['tier'] +' '+ Temp['rank']
    return Rank

In [292]:
Get_Rank('NA1',Get_Summoner_id('NA1','Doublelift'))

'DIAMOND I'

In [314]:
# Function gets the match details of the most recent match played by the summoner
def Last_Match(Region,Summoner):
    Summoner_Matches = LeagueWatcher.match.matchlist_by_puuid(Region, Get_Summoner_Info(Region,Summoner)['puuid'])
    Last_Match = Summoner_Matches[0]
    Match_Details = LeagueWatcher.match.by_id(Region,Last_Match)



    participants = []
    for row in Match_Details['info']['participants']:
        participants_row = {}
        participants_row['Summoner'] = row['summonerName']
        participants_row['Rank'] = Get_Rank(Region,Get_Summoner_id(Region,row['summonerName']))
        participants_row['champion'] = row['championName']
        participants_row['KDA'] = row['kills'] + row['assists'] / row['deaths']
        participants_row['Kills'] = row['kills']
        participants_row['Deaths'] = row['deaths']
        participants_row['Assists'] = row['assists']
        participants_row['CS']=(row['totalMinionsKilled']*60/row['timePlayed'])
        participants_row['Damage'] = row['totalDamageDealt']
        participants_row['Gold'] = row['goldEarned']
        participants_row['Level'] = row['champLevel']
        participants_row['Minions Killed'] = row['totalMinionsKilled']

        participants.append(participants_row)
    df = pd.DataFrame(participants)
    return df

In [315]:
Last_Match('NA1','Raiziel')

,Summoner,Rank,champion,KDA,Kills,Deaths,Assists,CS,Damage,Gold,Level,Minions Killed
0,Zero kuchiki,BRONZE IV,AurelionSol,30.000000,27,14,42,4.044503,150690,21768,18,103
1,icyyulises,Unranked,KogMaw,11.111111,9,18,38,1.727749,61708,15802,18,44
2,ProNinjaTeaY,GOLD IV,Lux,19.272727,14,11,58,2.670157,128079,18059,18,68
3,mko1,GOLD III,Zac,15.555556,9,9,59,3.141361,134184,18088,18,80
4,Raiziel,Unranked,Camille,27.133333,25,15,32,0.981675,74447,19441,18,25
5,Alpagon,Unranked,Blitzcrank,13.411765,12,17,24,1.099476,62774,16417,18,28
6,MeiTian,Unranked,Amumu,9.200000,7,15,33,1.413613,65776,15190,18,36
7,Mythical Mal,Unranked,Caitlyn,11.315789,10,19,25,3.337696,127472,16495,18,85
8,Meidri,Unranked,Karthus,26.200000,25,20,24,3.965969,157071,21296,18,101
9,AlpacaUUU,Unranked,Volibear,15.846154,13,13,37,0.942408,76312,17795,18,24
